In [1]:
import random
import networkx as nx

def seleccionar_inicio_y_final(grafo):
    # 1. Seleccionar un nodo de inicio aleatorio
    inicio = random.choice(list(grafo.nodes))

    # 2. Calcular las distancias desde el inicio a todos los demás nodos
    distancias = nx.single_source_dijkstra_path_length(grafo, inicio)

    # 3. Filtrar los nodos hoja (grado 1)
    hojas = [nodo for nodo in grafo.nodes if grafo.degree[nodo] == 1]

    # 4. Elegir entre las hojas la más lejana al nodo de inicio
    hoja_mas_lejana = max(hojas, key=lambda nodo: distancias.get(nodo, -1))

    return inicio, hoja_mas_lejana